In [31]:
from dataloader import LineMODCocoDataset
import torch

In [32]:
root = 'test_data_subset'
annFile = 'LOOKHEREannotations.json'

dataset = LineMODCocoDataset(root, annFile)

loading annotations into memory...
Done (t=0.06s)
creating index...
index created!


In [33]:
len(dataset)

18273

In [34]:
print(dataset.coco.dataset)

{'info': {'year': 2021, 'version': '1.0', 'description': 'BOP dataset converted to COCO format', 'date_created': '2021-01-01 00:00:00'}, 'images': [{'id': 1, 'width': 640, 'height': 480, 'file_name': '000001/rgb/000000.png'}, {'id': 2, 'width': 640, 'height': 480, 'file_name': '000001/rgb/000001.png'}, {'id': 3, 'width': 640, 'height': 480, 'file_name': '000001/rgb/000002.png'}, {'id': 4, 'width': 640, 'height': 480, 'file_name': '000001/rgb/000003.png'}, {'id': 5, 'width': 640, 'height': 480, 'file_name': '000001/rgb/000004.png'}, {'id': 6, 'width': 640, 'height': 480, 'file_name': '000001/rgb/000005.png'}, {'id': 7, 'width': 640, 'height': 480, 'file_name': '000001/rgb/000006.png'}, {'id': 8, 'width': 640, 'height': 480, 'file_name': '000001/rgb/000007.png'}, {'id': 9, 'width': 640, 'height': 480, 'file_name': '000001/rgb/000008.png'}, {'id': 10, 'width': 640, 'height': 480, 'file_name': '000001/rgb/000009.png'}, {'id': 11, 'width': 640, 'height': 480, 'file_name': '000001/rgb/000010

In [35]:
dataset[0]

(tensor([[[0.1804, 0.1922, 0.1882,  ..., 0.6157, 0.6196, 0.6235],
          [0.1725, 0.1922, 0.1647,  ..., 0.6235, 0.6235, 0.6118],
          [0.1725, 0.1647, 0.1804,  ..., 0.6353, 0.6275, 0.6118],
          ...,
          [0.2000, 0.1922, 0.2000,  ..., 0.3255, 0.3216, 0.3294],
          [0.2000, 0.1804, 0.1843,  ..., 0.3216, 0.3216, 0.3490],
          [0.1882, 0.2000, 0.1961,  ..., 0.3333, 0.3294, 0.3412]],
 
         [[0.1922, 0.1922, 0.2078,  ..., 0.6275, 0.6314, 0.6314],
          [0.1922, 0.1922, 0.1882,  ..., 0.6353, 0.6314, 0.6235],
          [0.1961, 0.1922, 0.1804,  ..., 0.6471, 0.6471, 0.6431],
          ...,
          [0.2039, 0.2078, 0.2275,  ..., 0.2941, 0.2941, 0.3137],
          [0.2157, 0.2235, 0.2196,  ..., 0.2863, 0.2941, 0.3098],
          [0.2039, 0.2392, 0.2196,  ..., 0.3137, 0.3098, 0.2941]],
 
         [[0.2314, 0.2353, 0.2314,  ..., 0.6824, 0.7020, 0.6784],
          [0.2196, 0.2118, 0.2235,  ..., 0.6863, 0.6824, 0.6863],
          [0.2235, 0.2000, 0.2196,  ...,